In [1]:
import azureml
from azureml.core import Workspace, Run
import pandas as pd
import numpy
from azureml.core.model import Model
ws = Workspace.from_config()

Found the config file in: C:\Users\zhpek\Desktop\C4TS-Challenge4\aml_config\config.json


In [2]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [3]:
ws.experiments

{'c4ts-challenge4': Experiment(Name: c4ts-challenge4,
 Workspace: pekamlws)}

In [8]:
experiment_name = 'c4ts-challenge4' 
experiment = ws.experiments[experiment_name]
#ml_run = AutoMLRun(experiment=experiment, run_id=run_id)
selected_metric = 'AUC_weighted'
#summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
#pattern = re.compile('^AutoML_[^_]*$')
runs = list(experiment.get_runs())
#model_dict = {}
models_df = pd.DataFrame(index = [selected_metric, 'run'])
for run in runs:
    
    aml_run = AutoMLRun(experiment = experiment, run_id = run.id)
    print(run.id)
    best_run, __ = aml_run.get_output()
    metrics = best_run.get_metrics()
    
    if selected_metric in metrics:
        #model_dict[run.id]= {'run': aml_run}
        #model_dict[run.id][selected_metric] = metrics[selected_metric]
        models_df[run.id] = [metrics[selected_metric],aml_run]
        
models_df = models_df.T

AutoML_2636c1c1-0631-4622-af27-41bab710af0c
AutoML_abe634ba-a22c-4ee3-a2d4-6f7596655277
AutoML_a0206893-ba6d-4d62-8bee-b7defa215a1a
AutoML_8a161d4a-5ab2-44e5-8471-20c323a1e0a6


In [9]:
models_df

,AUC_weighted,run
AutoML_2636c1c1-0631-4622-af27-41bab710af0c,0.796387,"Run(Experiment: c4ts-challenge4,\nId: AutoML_2..."
AutoML_abe634ba-a22c-4ee3-a2d4-6f7596655277,0.807797,"Run(Experiment: c4ts-challenge4,\nId: AutoML_a..."
AutoML_a0206893-ba6d-4d62-8bee-b7defa215a1a,0.805238,"Run(Experiment: c4ts-challenge4,\nId: AutoML_a..."
AutoML_8a161d4a-5ab2-44e5-8471-20c323a1e0a6,0.770865,"Run(Experiment: c4ts-challenge4,\nId: AutoML_8..."


In [10]:
run_id = models_df[selected_metric].astype(float).idxmax()
best_run = models_df.loc[run_id, 'run']
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
c4ts-challenge4,AutoML_abe634ba-a22c-4ee3-a2d4-6f7596655277,automl,Completed,Link to Azure Portal,Link to Documentation


In [11]:
model = best_run.register_model(description = experiment_name, tags = None)
model_id = best_run.model_id
model_id

Registering model AutoMLabe634baabest


'AutoMLabe634baabest'

In [12]:
rundl = AutoMLRun(experiment = experiment, run_id = best_run.id)
dependencies = rundl.get_run_sdk_dependencies(iteration = 2)

No issues found in the SDK package versions.


In [15]:
import pickle
from utils import scaler

artifacts = rundl.get_file_names()
print('Downloading files {}'.format(str(artifacts)))
for f in artifacts:
    rundl.download_file(f)

artifacts = ['pca_transform.pkl']
p = pickle.load(open('pca_transform.pkl','rb'))
s = pickle.load(open('scaler.pkl','rb'))
scaler_mean = list(s.mean)
scaler_var = list(s.var)
scaler_index = list(s.mean.index)

In [26]:
#%%writefile score_sparkml.py
#s_mean = {scaler_mean}
#s_var = {scaler_var}
#s_

#def scale(x):
#    result = (x - {scaler_mean}) / np.sqrt({scaler_var})
#    return (result)

score_script = """
 
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model
import pandas as pd
import numpy as np

def cleanLongLat(l):
    split = l.str.split(',', expand=True)
    split = (split[0]+'.'+[i if len(i)>1 else i+'0' for i in split[1]]).astype(float)
    return(split)

def scale(x):
    s_mean = pd.Series({scaler_mean},{scaler_index})
    s_var = pd.Series({scaler_var},{scaler_index})
    
    result = (x - s_mean) / np.sqrt(s_var)
    return (result)
    
def init():

    global model
    global pca_transform
    
    model_id = "{model_id}"
    model_path = Model.get_model_path(model_name = model_id)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    pca_transform = pickle.load(open('pca_transform.pkl', 'rb'))


def run(rawdata):
    try:
        df = json.loads(rawdata)['data']
        df = pd.DataFrame.from_records(df)
        df['Latitude'] = cleanLongLat(df['Latitude'])
        df['Longitude'] = cleanLongLat(df['Longitude'])
        df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
        df.drop('Failure_NextHour', axis = 1, inplace=True)
        df = df.apply(pd.to_numeric)
        data = pca_transform.transform(scale(df))[:,:10]
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({{"error": result}})
    return json.dumps({{"result":result.tolist()}}) 
    
""".format(model_id = model_id, scaler_mean = scaler_mean, scaler_var = scaler_var, scaler_index = scaler_index)#, scaler_mean = scaler_mean, scaler_var = scaler_var)

with open("score.py", "w") as file:
    file.write(score_script)

In [27]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','pandas'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

'myenv.yml'

In [28]:
%%time
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                                  execution_script = 'score.py',
                                                  conda_file = conda_env_file_name,
                                                  dependencies = artifacts,
                                                  description = "Image for c4ts Challenge 4")

image = Image.create(name = "c4tsimg2",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

if image.creation_state == 'Failed':
    print("Image build log at: " + image.image_build_log_uri)

Creating image
Running................................................
SucceededImage creation operation finished for image c4tsimg2:2, operation "Succeeded"
Wall time: 4min 24s


In [29]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "c4ts4",  "method" : "automl"}, 
                                               description='C4TS Challenge 4')

In [35]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_image(workspace=ws,
                                       name='c4ts-svc',
                                       deployment_config=aciconfig,
                                       image=image)

service.wait_for_deployment(show_output=True)
print(service.state)

Creating service
Running...........................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
Wall time: 2min 35s


In [36]:
print(service.get_logs())

2019-01-17T08:29:14,542560153+00:00 - rsyslog/run 
2019-01-17T08:29:14,543599154+00:00 - iot-server/run 
ok: run: gunicorn: (pid 14) 0s
ok: run: nginx: (pid 12) 0s
ok: run: rsyslog: (pid 11) 0s
2019-01-17T08:29:14,545007754+00:00 - nginx/run 
2019-01-17T08:29:14,547210855+00:00 - gunicorn/run 
ok: run: rsyslog: (pid 11) 0s
ok: run: rsyslog: (pid 11) 0s
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-01-17T08:29:14,836284276+00:00 - iot-server/finish 1 0
2019-01-17T08:29:14,837447176+00:00 - Exit code 1 is normal. Not restarting iot-server.
{"timestamp": "2019-01-17T08:29:15.085417Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-06dcb162b38d40c08a9d7399a86cb51c-13ca10e4aea3da2723ec04", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-01-17T08:

In [37]:
df = pd.read_csv('AssetData_Historical.csv')

In [38]:
import csv
import json
with open('AssetData_Historical.csv') as f:
    reader = csv.DictReader(f)
    titles = reader.fieldnames
    j_test = json.dumps({'data':[row for row in reader]})
    
result = service.run(input_data = j_test)

In [39]:
result

'{"result": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 